<a href="https://colab.research.google.com/github/ningxia202109/llm-learn/blob/main/python-langchain-tutorials/Ollama_llm_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Build a Simple LLM Application with LCEL**
https://python.langchain.com/v0.2/docs/tutorials/llm_chain/

In [1]:
!curl https://ollama.ai/install.sh | sh
!ollama serve > ollama-server.log 2>&1 &
!ollama run qwen2:0.5b > qwen2.log 2>&1 &
!sleep 120
!ollama list

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10975    0 10975    0     0  24054      0 --:--:-- --:--:-- --:--:-- 24067
>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
NAME      	ID          	SIZE  	MODIFIED           
qwen2:0.5b	6f48b936a09f	352 MB	About a minute ago	


In [2]:
!pip install -U -q langchain
!pip install -U -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.7 MB/s eta 0:00:00


In [5]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize an instance of the Ollama model
llm = Ollama(model="qwen2:0.5b")

# Using Language Models
messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="Today is a good day"),
]

# Invoke the model to generate responses
response = llm.invoke(messages)
print(response)

'今天是个好日子。'

In [6]:
# OutputParsers
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = llm.invoke(messages)
parser.invoke(result)

'今天是个好日子。'

In [7]:
# Chain

chain = llm | parser
chain.invoke(messages)


'今天是个好天'

In [15]:
# Prompt Templates
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

print(prompt_template)

input_variables=['language', 'text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate the following into {language}:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


In [16]:
# Format prompt
prompt = prompt_template.format(language="Chinese", text="Today is a good day")
print(prompt)

System: Translate the following into Chinese:
Human: Today is a good day


In [17]:
# Invoke prompt
result = prompt_template.invoke({"language": "Chinese", "text": "Today is a good day"})
print(result)
print(result.to_messages())

messages=[SystemMessage(content='Translate the following into Chinese:'), HumanMessage(content='Today is a good day')]
[SystemMessage(content='Translate the following into Chinese:'), HumanMessage(content='Today is a good day')]


In [18]:
# Chain prompt_template
chain = prompt_template | llm | parser
chain.invoke({"language": "Chinese", "text": "Today is a good day"})

'今天是个好日子'